In [ ]:
# Sujet :
# Prédire le pays gagnant lors des JO 2024 en terme de :
# nombre de médailles total ? points de médailles ?
# prédire le total de médailles de chaque pays et faire un classement


# relier host de result pour pouvoir exlure les jeux d'hiver

In [ ]:
# Import des biliothèques
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.multioutput import MultiOutputRegressor

In [ ]:
# Chargement des jeux de données
from sqlalchemy import create_engine

username = 'jo2024_fatima'
password = 'Ipssi2024!'
hostname = 'postgresql-jo2024.alwaysdata.net'
database = 'jo2024_postgres'

connection_url = f'postgresql://{username}:{password}@{hostname}:5432/{database}'

engine = create_engine(connection_url)

df_result = pd.read_sql("olympic_results", engine)
df_hosts = pd.read_sql("olympic_hosts", engine)

In [ ]:
df_result["country_name"].unique()

array(['Republic of Korea', 'Latvia', 'United States of America',
       'Russian Federation', 'Norway', 'Soviet Union', 'Greece',
       'Federal Republic of Germany', 'Sweden', 'Czech Republic',
       'Switzerland', 'German Democratic Republic (Germany)', 'Finland',
       'Czechoslovakia', 'Italy', 'Mexico', 'Argentina', 'Japan',
       'Independent Olympic Athletes', 'Canada', 'Germany',
       'Great Britain', 'Nigeria', 'Austria', 'Indonesia', 'Belgium',
       'Turkey', 'Slovenia', 'Netherlands', 'Australia', 'France',
       'Poland', "People's Republic of China", 'Denmark', 'ROC',
       'Kazakhstan', 'Portugal', 'Unified Team', 'Estonia', 'Jamaica',
       'New Zealand', 'Spain', 'Libya', 'Morocco', 'Egypt', 'Chile',
       'India', 'Islamic Republic of Iran', 'South Africa', 'Tunisia',
       'Saudi Arabia', 'Cuba', 'Peru', 'Hungary', 'Saar', 'Bulgaria',
       'Romania', 'Luxembourg', 'Bahrain', 'Kenya', 'Qatar', 'Bahamas',
       'Haiti', 'Belarus', 'Ukraine', 'Tajikistan

In [ ]:
#Merge des deux datasets
df_hosts.rename({'slug_game':'game_slug'}, axis=1, inplace=True)
df_result.rename({'slug_game':'game_slug'}, axis=1, inplace=True)
df_full = pd.merge(df_result, df_hosts, on='game_slug')


In [ ]:
# Nettoyage des données
df_full = df_full.dropna(subset=['country_name'])
df_full

,_c0,Unnamed: 0,discipline_title,event_title,game_slug,...,game_location,game_name,game_season,game_year,start_year
0,55739,55739,Skeleton,Individual men,salt-lake-city-2002,...,United States,Salt Lake City 2002,Winter,2002,2002
1,55740,55740,Skeleton,Individual men,salt-lake-city-2002,...,United States,Salt Lake City 2002,Winter,2002,2002
2,55741,55741,Skeleton,Individual men,salt-lake-city-2002,...,United States,Salt Lake City 2002,Winter,2002,2002
3,55742,55742,Skeleton,Individual men,salt-lake-city-2002,...,United States,Salt Lake City 2002,Winter,2002,2002
4,55743,55743,Skeleton,Individual men,salt-lake-city-2002,...,United States,Salt Lake City 2002,Winter,2002,2002
...,...,...,...,...,...,...,...,...,...,...,...
162799,162592,162592,Weightlifting,heavyweight - two hand lift men,athens-1896,...,Greece,Athens 1896,Summer,1896,1896
162800,162593,162593,Weightlifting,heavyweight - two hand lift men,athens-1896,...,Greece,Athens 1896,Summer,1896,1896
162801,162594,162594,Weightlifting,heavyweight - two hand lift men,athens-1896,...,Greece,Athens 1896,Summer,1896,1896
162802,162595,162595,Weightlifting,heavyweight - two hand lift men,athens-1896,...,Greece,Athens 1896,Summer,1896,1896


In [ ]:
summer_games_data = df_full[df_full['game_season'] == 'Summer']
summer_games_data

In [ ]:
# show duplicates
summer_games_data.duplicated().sum()

# drop duplicates
summer_games_data.drop_duplicates()

In [ ]:
summer_games_data.medal_type.unique()
summer_games_data.isnull().sum()

In [ ]:
# Remove useless columns
summer_games_data.drop(['Unnamed: 0', 'athlete_url', '_c0', 'country_3_letter_code', 'country_code', 'index', 'game_end_date', 'game_start_date', 'start_year', 'game_season', 'game_name'], axis=1, inplace=True)

In [ ]:
summer_games_data.columns

In [ ]:
summer_games_data

In [ ]:
game_part = summer_games_data.groupby(['country_name','game_year'])
print(game_part)

In [ ]:
summer_games_data.columns

In [ ]:
medals_data = summer_games_data[['country_name', 'medal_type', 'game_slug', 'athlete_full_name']]
medals_data

In [378]:
# Sélectionner les colonnes pertinentes
medals_data = summer_games_data[['country_name', 'medal_type', 'game_slug', 'athlete_full_name']]

# Extraire l'année des jeux olympiques
medals_data['year'] = medals_data['game_slug'].str.extract(r'(\d{4})').astype(int)

medals_data = medals_data[~medals_data['country_name'].isin(['Soviet Union', 'Unified Team'])]

# Dictionnaire correction
country_dict = {
    'German Democratic Republic (Germany)':'Germany',
    'Federal Republic of Germany':'Germany',
    "Democratic People's Republic of Korea":'North Korea',
    "ROC":"Russian Federation",
    "Republic of Korea":"South Korea",
    "People's Republic of China":'China',
    "Islamic Republic of Iran":'Iran',
    "United States of America":'USA'}

medals_data['country_name'] = medals_data['country_name'].replace(country_dict)

# Ajouter une colonne pour le nombre d'athlètes par pays par année
medals_data['num_athletes'] = medals_data.groupby(['country_name', 'year'])['athlete_full_name'].transform('nunique')

# Ajouter une colonne comptant les points de médailles
medals_data['medal_value'] = medals_data['medal_type'].map({'GOLD': 3, 'SILVER': 2, 'BRONZE': 1})

# Ajouter une colonne comptant le nombre de médailles
medals_data['medal_count'] = medals_data['medal_type'].map({'GOLD': 1, 'SILVER': 1, 'BRONZE': 1, 'None': 0})

# Agréger les données par pays et par année
medals_by_country_year = medals_data.groupby(['country_name', 'year', 'num_athletes'])[['medal_value', 'medal_count']].sum().reset_index()

# Renommer les colonnes pour plus de clarté
medals_by_country_year.columns = ['country_name', 'year', 'num_athletes', 'total_medal_value', 'total_medal_count']

<ipython-input-378-86349b662215>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  medals_data['year'] = medals_data['game_slug'].str.extract(r'(\d{4})').astype(int)


In [379]:
# Séparer les données d'entraînement (jusqu'à 2016) et de test (2020)
# train_data = medals_by_country_year[(medals_by_country_year['year'] >= 1990) & (medals_by_country_year['year'] <= 2016)]
train_data = medals_by_country_year[medals_by_country_year['year'] <= 2016]
test_data = medals_by_country_year[medals_by_country_year['year'] == 2020]

X_train = train_data[['country_name', 'year', 'num_athletes']]
y_train = train_data[['total_medal_value', 'total_medal_count']]
X_test = test_data[['country_name', 'year', 'num_athletes']]
y_test = test_data[['total_medal_value', 'total_medal_count']]

In [380]:
print(medals_by_country_year)

     country_name  year  num_athletes  total_medal_value  total_medal_count
0     Afghanistan  1936             0                0.0                0.0
1     Afghanistan  1948             0                0.0                0.0
2     Afghanistan  1956             0                0.0                0.0
3     Afghanistan  1960            10                0.0                0.0
4     Afghanistan  1964             8                0.0                0.0
...           ...   ...           ...                ...                ...
2870     Zimbabwe  2004             5                6.0                3.0
2871     Zimbabwe  2008            10                9.0                4.0
2872     Zimbabwe  2012             7                0.0                0.0
2873     Zimbabwe  2016            11                0.0                0.0
2874     Zimbabwe  2020             2                0.0                0.0

[2875 rows x 5 columns]


# Linear Regression

In [385]:
# Préparer le transformateur pour l'encodage et la normalisation
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['year', 'num_athletes']),
    ])

# Créer le pipeline avec le préprocesseur et le modèle de régression linéaire
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    # On prédit à la fois le nombre de points de médailles et le nombre médaille avec la régression linéaire multi-sorties
    ('regressor', MultiOutputRegressor(LinearRegression()))
])

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédire les médailles pour 2020
y_pred = model.predict(X_test)

# Évaluer le modèle
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Afficher les prédictions
predictions = pd.DataFrame({
    'country_name': X_test['country_name'],
    'predicted_points': y_pred[:, 0],
    'predicted_medals': y_pred[:, 1]
})

# Remplacer les valeurs négatives par 0
predictions['predicted_points'] = predictions['predicted_points'].apply(lambda x: max(0, x))
predictions['predicted_medals'] = predictions['predicted_medals'].apply(lambda x: max(0, x))

# Classer la dataframe dans l'ordre des médailles gagnées par pays
predictions = predictions.sort_values(by='predicted_points', ascending=False)

print("total medals in 2020 : " + str(predictions['predicted_medals'].sum()))
print(predictions)

Mean Absolute Error: 6.7018541473744495
total medals in 2024 : 1067.4272483366617
                          country_name  predicted_points  predicted_medals
2703                               USA        165.211630         82.356017
1393                             Japan        114.747639         57.252790
545                              China        104.306814         52.059019
2199                Russian Federation        100.826539         50.327762
960                            Germany         92.705897         46.288162
911                             France         92.705897         46.288162
151                          Australia         90.385713         45.133990
1005                     Great Britain         88.065530         43.979819
1341                             Italy         86.905438         43.402733
473                             Canada         71.244200         35.612077
2424                             Spain         63.123557         31.572477
2393              

In [386]:
# Prédiction pour l'année 2024

# Estimer le nombre d'athlètes pour 2024
avg_athletes = medals_by_country_year.groupby('country_name')['num_athletes'].mean().reset_index()
avg_athletes['year'] = 2024

# Créer les données de prédiction pour 2024
X_2024 = avg_athletes[['country_name', 'year', 'num_athletes']]

# Prédire les médailles pour 2024
y_2024_pred = model.predict(X_2024)

# Afficher les prédictions
predictions = pd.DataFrame({
    'country_name': X_2024['country_name'],
    'predicted_points': y_2024_pred[:, 0],
    'predicted_medals': y_2024_pred[:, 1]
})

# Remplacer les valeurs négatives par 0
predictions['predicted_points'] = predictions['predicted_points'].apply(lambda x: max(0, x))
predictions['predicted_medals'] = predictions['predicted_medals'].apply(lambda x: max(0, x))

# Classer la dataframe dans l'ordre des médailles gagnées par pays
predictions = predictions.sort_values(by='predicted_points', ascending=False)

print("total medals in 2024 : " + str(predictions['predicted_medals'].sum()))
print(predictions)

total medals in 2024 : 763.7511601029194
           country_name  predicted_points  predicted_medals
211                 USA        110.432574         55.111879
168  Russian Federation         85.565179         42.741635
76              Germany         81.611943         40.775105
213             Ukraine         70.948023         35.470356
42                China         67.244653         33.628121
..                  ...               ...               ...
100             Jamaica          0.000000          0.000000
102              Jordan          0.000000          0.000000
105            Kiribati          0.000000          0.000000
106              Kosovo          0.000000          0.000000
230            Zimbabwe          0.000000          0.000000

[231 rows x 3 columns]


# Polynomial Regression

In [383]:
# Préparer le transformateur pour l'encodage et la normalisation
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['year', 'num_athletes']),
    ])

polynomial_features = PolynomialFeatures(degree=2, include_bias=False)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('polynomial_features', polynomial_features),
    ('regressor', MultiOutputRegressor(LinearRegression()))
])

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédire les médailles pour 2020
y_pred = model.predict(X_test)

# Évaluer le modèle
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Afficher les prédictions
predictions = pd.DataFrame({
    'country_name': X_test['country_name'],
    'predicted_points': y_pred[:, 0],
    'predicted_medals': y_pred[:, 1]
})

# Remplacer les valeurs négatives par 0
predictions['predicted_points'] = predictions['predicted_points'].apply(lambda x: max(0, x))
predictions['predicted_medals'] = predictions['predicted_medals'].apply(lambda x: max(0, x))

# Classer la dataframe dans l'ordre des médailles gagnées par pays
predictions = predictions.sort_values(by='predicted_points', ascending=False)

print("total medals : " + str(predictions['predicted_medals'].sum()))
print(predictions)

Mean Absolute Error: 4.162259762913595
total medals : 960.5574942296804
                          country_name  predicted_points  predicted_medals
2703                               USA        210.934847        103.671162
1393                             Japan        122.259297         60.658603
545                              China        106.614262         53.030234
2199                Russian Federation        101.605088         50.584266
911                             France         90.317258         45.065271
960                            Germany         90.317258         45.065271
151                          Australia         87.195083         43.536826
1005                     Great Britain         84.118650         42.029896
1341                             Italy         82.597586         41.284500
473                             Canada         63.182477         31.748119
2424                             Spain         53.935878         27.189269
2393                       S

# Random Forest

In [384]:
# Préparer le transformateur pour l'encodage et la normalisation
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), ['year', 'num_athletes']),
    ])

polynomial_features = PolynomialFeatures(degree=2, include_bias=False)
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('polynomial_features', polynomial_features),
    ('regressor', MultiOutputRegressor(RandomForestRegressor(random_state=42)))
])

# Entraîner le modèle
model.fit(X_train, y_train)

# Prédire les médailles pour 2020
y_pred = model.predict(X_test)

# Évaluer le modèle
mae = mean_absolute_error(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')

# Afficher les prédictions
predictions = pd.DataFrame({
    'country_name': X_test['country_name'],
    'predicted_points': y_pred[:, 0],
    'predicted_medals': y_pred[:, 1]
})

# Remplacer les valeurs négatives par 0
predictions['predicted_points'] = predictions['predicted_points'].apply(lambda x: max(0, x))
predictions['predicted_medals'] = predictions['predicted_medals'].apply(lambda x: max(0, x))

# Classer la dataframe dans l'ordre des médailles gagnées par pays
print("total medals : " + str(predictions['predicted_medals'].sum()))
print(predictions.sort_values(by='predicted_medals', ascending=False))

Mean Absolute Error: 3.1483452596045143
total medals : 940.3089420338069
                          country_name  predicted_points  predicted_medals
2703                               USA        235.770000        113.290000
1393                             Japan        139.330000         68.240000
545                              China         96.450000         48.470000
2199                Russian Federation         96.470000         48.040000
151                          Australia        102.090000         46.120000
911                             France         85.860000         39.100000
960                            Germany         85.860000         39.100000
1005                     Great Britain         75.870000         37.880000
1341                             Italy         68.200000         34.520000
473                             Canada         56.155000         31.160000
1834                       Netherlands         34.940000         16.850000
378                        